If you are interested in UFC and you want to know how the american betting odds are working, this notebook might be helpful to you ! 

And hopefully it brings some interesting insight to everyone!

In [ ]:
#imports 
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

<img src="https://www.thenationalnews.com/image/policy:1.1054478:1602128191/GettyImages-1258439122.jpg?$p=9b4f9dd&w=2048&$w=dbf56c5" length="100px" width="1000px">

In [ ]:
# Data available
os.listdir('../input/ultimate-ufc-dataset')

In [ ]:
df = pd.read_csv('../input/ultimate-ufc-dataset/ufc-master.csv')
df.head()

### Initial Data Exploration

In [ ]:
data = df[~df.weight_class.str.contains("Catch Weight")]

weight_class_order = ["Women's Strawweight", "Women's Flyweight", "Women's Bantamweight", "Women's Featherweight",
                      "Flyweight", "Bantamweight", "Featherweight", "Lightweight", "Welterweight", 
                      "Middleweight", "Light Heavyweight", "Heavyweight"]

data['ordered_weight_class'] = pd.Categorical(data['weight_class'], weight_class_order)

In [ ]:
sns.countplot(data=data.sort_values('ordered_weight_class', ascending=False) , y = 'ordered_weight_class' , hue ='gender')
plt.xlabel('Number of Fights')
plt.ylabel('Ordered Weight Class')
plt.show();

#### NB:
Catch_weight is also present in the dataset, but it is not a weight category. When fighters fail to cut enough weight to make it in the weight division they agreed to fight in, an offer to fight at catch weight is made, so both fighters can't still fight. It allows both fighters to have some extra pound (128 instead of 125 for instance).

In [ ]:
sns.countplot(data=data.sort_values('finish') , y = 'finish')
plt.xlabel('finish type')

If the fight drags on past the number of allocated rounds (3 or 5), the judges decide who wins. If all three judges agree, it's a unanimous decision (U-DEC). If only 2 judges agree, it's a split decision (S-DEC).

In [ ]:
df["finish_details"].unique()

In [ ]:
# Using Label Encoder to make it more lisible on x axis
label = LabelEncoder()
df["finish_details"] = df["finish_details"].fillna('0')
label.fit(df['finish_details'])
label_name_mapping = dict(zip(label.classes_, label.transform(label.classes_)))
df["finish_details"] = label.transform(df["finish_details"])
plt.figure(figsize=(30, 30))
finish_det = df['finish_details'].loc[df['finish_details'] != 0]
sns.countplot(finish_det)
plt.show()

In [ ]:
print(label_name_mapping)

For the type of finish a lot of the data is unavailable (labeled zero), however for the amount of data we have we can clearly see that punches, and rear naked choke are significantly common for finishing strikes. Guillotine chokes is also a counter you can see in different matches.

In [ ]:
sns.countplot(df['finish_round'])
plt.xlabel("finish round")

In [ ]:
df['total_fight_time_secs'].fillna(0)

Looks like there is a mistake in the reach_dif feature, as the minimum seems way to low, we will clip it in the plot.

In [ ]:
#sns.set_theme(style="whitegrid")

# Load the example planets dataset
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=df,
    x="total_fight_time_secs", y="reach_dif",
    hue="age_dif", 
    size="weight_class",
    palette=cmap, sizes=(5, 100),
)
g.ax.xaxis.grid(True, "minor", linewidth=.1)
g.ax.yaxis.grid(True, "minor", linewidth=.1)
plt.ylim(1, 12)
plt.xlim(1, 1600)
g.despine(left=True, bottom=True)
plt.show()

As we could expect, big age gap with small reach difference usually end up in a quick victory. Also, most fight are three rounds resulting in the vertical line at 900 seconds.

In [ ]:
# There is a quick catch in the data for the countries where there is a white space for most the countries,
# Here is a quick fix, You might need to fix this if you want to use a model to predict betting odds
df['country'].unique()

In [ ]:
# Quick fix
cond = (df['country'].str[0] != ' ')
df['country_fixed'] = df['country'].where(cond, df['country'].str[1:])

In [ ]:
n_countries = len(df['country_fixed'].unique())
labels = list(df['country_fixed'].unique())
size = df['country_fixed'].value_counts()
colors = plt.cm.Wistia(np.linspace(0, 1, len(df['country_fixed'].unique())))
explode = [0 for i in range(n_countries)]
explode[0] = 0.05

plt.figure(figsize=(12, 12))
plt.pie(size, labels = labels, colors = colors, explode = explode, shadow = True, startangle = 90)
plt.title('The UFC has a big domestic market that accounts for more than half of the fights', fontsize = 25)
plt.legend([labels[:3]])
plt.show()

### Who is the winner ? 

In [ ]:
# all_fighters = pd.concat([df['R_fighter'], df['B_fighter']]).unique()
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
sns.countplot(ax=axes[0], data=data.sort_values('Winner'), y = 'Winner')

plt.xlabel('Number of Fights')
axes[0].set_title("Overall winner")
title_bouts = df[df['title_bout'] == True]
red_winner = title_bouts["Winner"][title_bouts['Winner'] == "Red"].count() / title_bouts["Winner"].count() * 100
blue_winner = title_bouts["Winner"][title_bouts['Winner'] == "Blue"].count() / title_bouts["Winner"].count() * 100
sns.countplot(ax=axes[1], data=title_bouts.sort_values("Winner"), y = "Winner")
plt.xlabel("number of title bouts")
axes[1].set_title("Winner of title bout")
non_title_bouts = df[df['title_bout'] == False]
red_winner = non_title_bouts["Winner"][non_title_bouts['Winner'] == "Red"].count() / non_title_bouts["Winner"].count() * 100
blue_winner = non_title_bouts["Winner"][non_title_bouts['Winner'] == "Blue"].count() / non_title_bouts["Winner"].count() * 100
sns.countplot(ax=axes[2], data=non_title_bouts.sort_values("Winner"), y = "Winner")
plt.xlabel("number of non title bouts")
plt.title("non title bout")
axes[1].set_title("Winner of title bout")

In [ ]:
red_winner = df["Winner"][df['Winner'] == "Red"].count() / df["Winner"].count() * 100
blue_winner = df["Winner"][df['Winner'] == "Blue"].count() / df["Winner"].count() * 100
print(f"For non-title bouts, {red_winner:.2f}% of the time the winner is red, against {blue_winner:.2f}% for the blue corner")
title_bouts = df[df['title_bout'] == True]
red_winner = title_bouts["Winner"][title_bouts['Winner'] == "Red"].count() / title_bouts["Winner"].count() * 100
blue_winner = title_bouts["Winner"][title_bouts['Winner'] == "Blue"].count() / title_bouts["Winner"].count() * 100
print(f"For title bouts, {red_winner:.2f}% of the time the winner is red against, {blue_winner:.2f}%")

Here, it is important to notice that the champion usually comes out in the red corner. This represent a big bias in the data.

# So what about the odds ? 

In [ ]:
sns.distplot(df["B_odds"])

In [ ]:
sns.distplot(df["R_odds"])

The Red corner seems to have safer bets, indeed once again the champion tends to come out of the red corner.

#### Notes on betting odds:

   In UFC there is a lot of options to bet money, the odds shown here is for the fight outcome. You can also bet on the method of submission and how long the fight will last.
    
   The Red corner seems to have safer bets (Minus sign, favourite). Indeed the favourite or the champion usually comes out second from the red corner. 

# Find the burning contenders

We can now try to plot the graph showing who fought who. We could plot a graph in each weight category, but we will focus here only on lightweight and welterweight. The two most populated weight categories.

In [ ]:
all_fighters = pd.concat([df['R_fighter'], df['B_fighter']]).unique()

In [ ]:
def make_fighters_dict(df):
    fighters_summary = dict()
    for name in all_fighters:
        total_fights = df["Winner"][df["R_fighter"] == name].count() + df["Winner"][df["B_fighter"] == name].count()
        wins = df["Winner"][df["Winner"] == "Red"][df["R_fighter"] == name].count() + df["Winner"][df["Winner"] == "Blue"][df["B_fighter"] == name].count()
        losses = total_fights - wins
        fighters_summary[name] = (total_fights, wins, losses)
    return fighters_summary

fighters_summary = make_fighters_dict(df)

In [ ]:
array_fighters = np.array(list(fighters_summary.values()))

In [ ]:
fighters_df = pd.DataFrame({'name':fighters_summary.keys(), 'total_bouts': array_fighters[:, 0], 'K':array_fighters[:, 1], 'D': array_fighters[:, 2]})
fighters_df.describe()

In [ ]:
def find_main_fighters(fighters_summary, threshold=16, categories=["all"]):
    if categories == ["all"]:
        main_fighters = {name: t for name, t in fighters_summary.items() if t[0] > threshold}
        print(f"There is {len(main_fighters)} with strictly more than {threshold} fights")
        return main_fighters
    else:
        main_fighters = {}
        for name, t in fighters_summary.items():
            if t[0] > threshold:
                if (df[df["R_fighter"] == name]["weight_class"].any() 
                    or df[df["R_fighter"] == name]["weight_class"].any())in categories:
                    main_fighters[name] = t
        print(f"There is {len(main_fighters)} fighters with strictly more than {threshold} fights in the categories {categories}")
        
    return main_fighters

main_fighters = find_main_fighters(fighters_summary, categories=["Lightweight", "Welterweight"])

In [ ]:
def make_graph(df, fighters_summary):
    G = nx.Graph()
    G.add_nodes_from(list(fighters_summary.items()))
    for i, (name1, name2) in enumerate(zip(df["R_fighter"], df["B_fighter"])):
        names = [name[0] for name in list(G.nodes())]
        if name1 in names and name2 in names:
            G.add_edge(name1, name2)
    return G
        

In [ ]:
G = make_graph(df, main_fighters)
isolated_nodes = [nodes for nodes in nx.isolates(G)]
while len(isolated_nodes) > 0:
    G.remove_node(isolated_nodes.pop())

In [ ]:
pos=nx.spring_layout(G)
plt.figure(figsize=(20, 20))
nx.draw_networkx_nodes(G, pos=pos, node_size=1000, label=G.nodes, cmap=plt.cm.Blues)
nx.draw_networkx_edges(G, pos=pos)
nx.draw_networkx_labels(G, pos=pos)
plt.show()

If you are watching UFC you are probably noticing a couple of familiar names!

If you enjoyed this notebook, please consider upvoting this notebooks and the Kaggle Dataset.

### More might come: 
- More data explanatory analysis
- more fights prediction with Graph Neural Network
- And more ! 